# DSC 10 Discussion Week 4
---

Welcome to Discussion 4!

Here's a quick rundown of topics for this week:

- **Merge**: "cross-reference" tables - remember the coffee example from class?

    - *(fun fact : TA Luke was the manager at Perks Coffee Shop before it closed two years ago)*

- **GroupBy** : group data entries together by column value - we can also specify multiple columns!

- **Conditionals** : if statements

- **Iteration** : for loops

- **Simulation** : probability experiments

You can find additional help on these topics in the course [textbook](https://eldridgejm.github.io/dive_into_data_science/front.html).

[Here](https://ucsd-ets.github.io/dsc10-2020-fa/published/default/reference/babypandas-reference.pdf) is a pointer to that reference sheet we saw last time.

<img src="data/panda_tree.jpg" width="1000">


In [37]:
import babypandas as bpd
import numpy as np

import otter
grader = otter.Notebook()

from notebook.services.config import ConfigManager

cm = ConfigManager()
cm.update(
    "livereveal", {
        'width': 1500,
        'height': 700,
        "scroll": True,
})

{'width': 1500, 'height': 700, 'scroll': True}

# College Scorecard

### A wild dataset has appeared!

Check out some interactions with the dataset here --> http://collegescorecard.ed.gov

In [2]:
colleges = bpd.read_csv('data/csc_basic.csv').set_index('UNITID')
colleges

,INSTNM,CITY,STABBR,PREDDEG,CONTROL,UGDS,RELAFFIL,DISTANCEONLY,UGDS_NRA,PCTPELL,PCTFLOAN
UNITID,,,,,,,,,,,
100654,Alabama A & M University,Normal,AL,3,1,4616.0,0,0.0,0.0065,0.7039,0.7667
100663,University of Alabama at Birmingham,Birmingham,AL,3,1,12047.0,0,0.0,0.0179,0.3525,0.5179
100690,Amridge University,Montgomery,AL,3,2,293.0,1,1.0,0.0000,0.6971,0.8436
100706,University of Alabama in Huntsville,Huntsville,AL,3,1,6346.0,0,0.0,0.0303,0.2949,0.4312
100724,Alabama State University,Montgomery,AL,3,1,4704.0,0,0.0,0.0159,0.7815,0.8113
...,...,...,...,...,...,...,...,...,...,...,...
489830,Arthur's Beauty College-Jonesboro,Jonesboro,AR,1,3,71.0,0,0.0,0.0000,0.7525,0.7228
489900,Palm Beach Academy of Health & Beauty-Distinct...,Lauderhill,FL,1,3,27.0,0,0.0,0.0000,0.0000,0.0000
489937,Piedmont International University,Winston-Salem,NC,3,2,295.0,1,0.0,0.0034,0.3634,0.3343


# Question 1.1 (review)

Which state has the most colleges/universities? (save the state abbreviation)

<!--
BEGIN QUESTION
name: q11
-->

In [3]:
# BEGIN SOLUTION
state_most_colleges = ( 
    colleges
    .groupby('STABBR')
    .count()
    .sort_values(by='CITY')
    .get(['CITY'])
    .index[-1] 
)
# END SOLUTION
state_most_colleges

'CA'

# State Population

**Another wild dataset has appeared!**

We'll need this data for the next question.

In [4]:
pops = bpd.read_csv('data/state-population.csv')
pops

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0
...,...,...,...,...
2539,USA,total,2010,309326295.0
2540,USA,under18,2011,73902222.0
2541,USA,total,2011,311582564.0
2542,USA,under18,2012,73708179.0


# Question 1.2

Which state has the largest number of colleges *per person*?

HINT: Play around with the old dataset and this new dataset then use ```.merge()``` 

<!--
BEGIN QUESTION
name: q12
-->

In [5]:
# get a DataFrame of colleges_per_state
colleges_per_state = colleges.groupby('STABBR').count().get(['CITY']) # SOLUTION
colleges_per_state

,CITY
STABBR,
AK,9
AL,89
AR,80
AS,1
AZ,116
...,...
VT,22
WA,108
WI,94


In [6]:
# get a DataFrame of pops_by_state
# BEGIN SOLUTION
pops_by_state = (
    pops
    [(pops.get('ages') == 'total') & (pops.get('year') == 2012)] 
    .get(["state/region","population"])
)
# END SOLUTION
pops_by_state

,state/region,population
1,AL,4817528.0
95,AK,730307.0
97,AZ,6551149.0
191,AR,2949828.0
193,CA,37999878.0
...,...,...
2304,WV,1856680.0
2399,WI,5724554.0
2400,WY,576626.0
2495,PR,3651545.0


In [7]:
# now merge them!
# BEGIN SOLUTION
colleges_with_pops = (
    colleges_per_state
    .merge(pops_by_state, left_index=True, right_on='state/region') 
)
# END SOLUTION
colleges_with_pops

,CITY,state/region,population
95,9,AK,730307.0
1,89,AL,4817528.0
191,80,AR,2949828.0
97,116,AZ,6551149.0
193,607,CA,37999878.0
...,...,...,...
2207,22,VT,625953.0
2303,108,WA,6895318.0
2399,94,WI,5724554.0
2304,73,WV,1856680.0


In [8]:
# calculate a new per_person column
# BEGIN SOLUTION
per_person = colleges_with_pops.assign(per_person = colleges_with_pops.get('CITY') / colleges_with_pops.get('population'))
# END SOLUTION
per_person

,CITY,state/region,population,per_person
95,9,AK,730307.0,0.000012
1,89,AL,4817528.0,0.000018
191,80,AR,2949828.0,0.000027
97,116,AZ,6551149.0,0.000018
193,607,CA,37999878.0,0.000016
...,...,...,...,...
2207,22,VT,625953.0,0.000035
2303,108,WA,6895318.0,0.000016
2399,94,WI,5724554.0,0.000016
2304,73,WV,1856680.0,0.000039


In [9]:
# BEGIN SOLUTION
largest_colleges_per_person = (
    per_person
    .sort_values(by='per_person')
    .get("state/region")
    .iloc[-1]
)
# END SOLUTION
largest_colleges_per_person

'ND'

# Question 1.3

What if we had set the index of `pops_by_state`?

<!--
BEGIN QUESTION
name: q13
-->

In [10]:
pops_by_state_with_index = pops_by_state.set_index('state/region') # SOLUTION
pops_by_state_with_index

,population
state/region,
AL,4817528.0
AK,730307.0
AZ,6551149.0
AR,2949828.0
CA,37999878.0
...,...
WV,1856680.0
WI,5724554.0
WY,576626.0


In [11]:
# merge this with colleges_by_state

# BEGIN SOLUTION
colleges_with_pops_by_index = (
    colleges_per_state
    .merge(pops_by_state_with_index, left_index=True, right_index=True) 
)
# END SOLUTION
colleges_with_pops_by_index

,CITY,population
AK,9,730307.0
AL,89,4817528.0
AR,80,2949828.0
AZ,116,6551149.0
CA,607,37999878.0
...,...,...
VT,22,625953.0
WA,108,6895318.0
WI,94,5724554.0
WV,73,1856680.0


In [12]:
# calculate a new per_person column

# BEGIN SOLUTION
per_person_with_index = (
    colleges_with_pops_by_index.assign(per_person = colleges_with_pops_by_index.get('CITY') / colleges_with_pops_by_index.get('population'))
)
# END SOLUTION
per_person_with_index

,CITY,population,per_person
AK,9,730307.0,0.000012
AL,89,4817528.0,0.000018
AR,80,2949828.0,0.000027
AZ,116,6551149.0,0.000018
CA,607,37999878.0,0.000016
...,...,...,...
VT,22,625953.0,0.000035
WA,108,6895318.0,0.000016
WI,94,5724554.0,0.000016
WV,73,1856680.0,0.000039


In [13]:
# BEGIN SOLUTION
largest_colleges_per_person_with_index = (
    per_person_with_index
    .sort_values(by='per_person')
    .index[-1]
)
# END SOLUTION
largest_colleges_per_person_with_index

'ND'

# Question 1.4

The `preddeg` column specifies the primary degree type offered (Associates, Bachelors, etc.) How many institutions of each type does each state have?

<!--
BEGIN QUESTION
name: q14
-->

In [14]:
count_institute_type = colleges.groupby(['STABBR', 'PREDDEG']).count() # SOLUTION
count_institute_type

INSTNM  CITY  CONTROL  UGDS  RELAFFIL  DISTANCEONLY  UGDS_NRA  \
STABBR PREDDEG                                                                  
AK     1             5     5        5     5         5             5         5   
       2             1     1        1     1         1             1         1   
       3             3     3        3     3         3             3         3   
AL     1            31    31       31    31        31            31        31   
       2            21    21       21    21        21            21        21   
...                ...   ...      ...   ...       ...           ...       ...   
WV     2            16    16       16    16        16            16        16   
       3            21    21       21    21        21            21        21   
WY     1             2     2        2     2         2             2         2   
       2             8     8        8     8         8             8         8   
       3             1     1        1     1         1             1         1   

                PCTPELL  PCTFLOAN  
STABBR PREDDEG                     
AK     1              5         5  
       2              1         1  
       3              3         3  
AL     1             31        31  
       2             21        21  
...                 ...       ...  
WV     2             16        16  
       3             21        21  
WY     1              2         2  
       2              8         8  
       3              1         1  

[163 rows x 9 columns]

In [ ]:
grader.check("q14")

# Question 1.5

Suppose that a college is considered large if it has more than 15000 undergrads, "medium" if it has more than 5000 but <= 15000, "small" if it has more than 100 but <= 5000, and "tiny" if it has <= than 100 students. Write a function `college_size` which takes in a number of undergrads and returns a string ("tiny", "small", "medium", "large").


<!--
BEGIN QUESTION
name: q15
-->

In [16]:
def college_size(n): # SOLUTION NO PROMPT
    if n <= 100: # SOLUTION NO PROMPT
        return 'tiny' # SOLUTION NO PROMPT
    elif n <= 5_000: # SOLUTION NO PROMPT
        return 'small' # SOLUTION NO PROMPT
    elif n <= 15_000: # SOLUTION NO PROMPT
        return 'medium' # SOLUTION NO PROMPT
    else: # SOLUTION NO PROMPT
        return 'large' # SOLUTION NO PROMPT

In [ ]:
grader.check("q15")

# College Scorecard with Earnings

### What? The dataset is evolving!

Here we see a bit more info about all of our colleges.

In [18]:
with_earnings = bpd.read_csv('data/csc_financials.txt')
with_earnings

,UNITID,INSTNM,CITY,STABBR,PREDDEG,CONTROL,UGDS,RELAFFIL,DISTANCEONLY,UGDS_NRA,PCTPELL,PCTFLOAN,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP,RPY_3YR_RT_SUPP,GT_25K_P6,NPT4
0,100654,Alabama A & M University,Normal,AL,3,1,4616.0,0,0.0,0.0065,0.7039,0.7667,29900,35000.0,0.245850,0.453,15567.0
1,100663,University of Alabama at Birmingham,Birmingham,AL,3,1,12047.0,0,0.0,0.0179,0.3525,0.5179,40200,21500.0,0.519911,0.669,16475.0
2,100690,Amridge University,Montgomery,AL,3,2,293.0,1,1.0,0.0000,0.6971,0.8436,40100,23000.0,0.233100,0.658,10155.0
3,100706,University of Alabama in Huntsville,Huntsville,AL,3,1,6346.0,0,0.0,0.0303,0.2949,0.4312,45600,23500.0,0.549003,0.685,19423.0
4,100724,Alabama State University,Montgomery,AL,3,1,4704.0,0,0.0,0.0159,0.7815,0.8113,26700,32091.0,0.196354,0.393,15037.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6294,489760,Milan Institute-Bakersfield West,Bakersfield,CA,2,3,78.0,0,0.0,0.0000,0.6506,0.6265,-1,-1.0,-1.000000,-1.000,23291.0
6295,489812,Pima Medical Institute-Dillon,Dillon,MT,1,3,35.0,0,0.0,0.0000,0.1053,0.2632,-1,-1.0,-1.000000,-1.000,18671.0
6296,489830,Arthur's Beauty College-Jonesboro,Jonesboro,AR,1,3,71.0,0,0.0,0.0000,0.7525,0.7228,-1,-1.0,-1.000000,-1.000,10263.0
6297,489937,Piedmont International University,Winston-Salem,NC,3,2,295.0,1,0.0,0.0034,0.3634,0.3343,-1,-1.0,-1.000000,-1.000,16669.0


# Question 1.6

What is the median earnings `MD_EARN_WNE_P10` amount for UCSD graduates?


<!--
BEGIN QUESTION
name: q16
-->

In [19]:
# BEGIN SOLUTION
median_earnings_UCSD = (
    with_earnings
    .set_index('INSTNM')
    .get('MD_EARN_WNE_P10')
    .loc['University of California-San Diego']
)
# END SOLUTION

median_earnings_UCSD

58600

In [ ]:
grader.check("q16")

# Part 2 : Cards

The next few questions will be about a standard deck of cards

In [21]:
# Create a deck of cards as a list

values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A']
suits = ['hearts', 'diamonds', 'clubs', 'spades']

ALL_CARDS = []
for value in values:
    for suit in suits:
        card = str(value) + ' of ' + suit
        ALL_CARDS.append(card)

In [22]:
ALL_CARDS

['2 of hearts',
 '2 of diamonds',
 '2 of clubs',
 '2 of spades',
 '3 of hearts',
 '3 of diamonds',
 '3 of clubs',
 '3 of spades',
 '4 of hearts',
 '4 of diamonds',
 '4 of clubs',
 '4 of spades',
 '5 of hearts',
 '5 of diamonds',
 '5 of clubs',
 '5 of spades',
 '6 of hearts',
 '6 of diamonds',
 '6 of clubs',
 '6 of spades',
 '7 of hearts',
 '7 of diamonds',
 '7 of clubs',
 '7 of spades',
 '8 of hearts',
 '8 of diamonds',
 '8 of clubs',
 '8 of spades',
 '9 of hearts',
 '9 of diamonds',
 '9 of clubs',
 '9 of spades',
 '10 of hearts',
 '10 of diamonds',
 '10 of clubs',
 '10 of spades',
 'J of hearts',
 'J of diamonds',
 'J of clubs',
 'J of spades',
 'Q of hearts',
 'Q of diamonds',
 'Q of clubs',
 'Q of spades',
 'K of hearts',
 'K of diamonds',
 'K of clubs',
 'K of spades',
 'A of hearts',
 'A of diamonds',
 'A of clubs',
 'A of spades']

# Question 2.1

Simulate drawing 5 cards *without replacement*

<!--
BEGIN QUESTION
name: q21
-->

In [23]:
np.random.choice(ALL_CARDS, 5, replace=False) # SOLUTION

array(['K of clubs', '5 of spades', '6 of spades', 'A of spades',
       'J of spades'], dtype='<U14')

# Question 2.2

Simulate drawing 5 cards *with* replacement
<!--
BEGIN QUESTION
name: q22
-->

In [24]:
np.random.choice(ALL_CARDS, 5, replace=True) # SOLUTION

array(['6 of spades', '7 of spades', 'J of diamonds', '4 of hearts',
       '9 of spades'], dtype='<U14')

# Question 2.3

Make a function ```is_card_clubs(card)``` which, given a single card, determines if that card is a club.

<!--
BEGIN QUESTION
name: q11
-->

In [25]:
def is_card_clubs(card):
# BEGIN SOLUTION
    card_is_clubs = card.endswith("clubs")
    return card_is_clubs

def is_card_clubs_other(card):
    card_is_clubs = ("clubs" in card)
    return card_is_clubs

card = np.random.choice(ALL_CARDS, 1, replace=True)[0] # draw a random card
# END SOLUTION

In [ ]:
grader.check("q11")

# Question 2.4

Make a function `number_of_suit(cards, suit)` which, given a list of cards, counts the number of cards matching the suit.

<!--
BEGIN QUESTION
name: q24
-->

In [27]:
def number_of_suit(cards, suit):
# BEGIN SOLUTION    
    number = 0
    for card in cards:
        if card.endswith(suit):
            number = number + 1
    return number
# END SOLUTION

# Question 2.5

What is the probability that a hand of 5 cards, drawn *without* replacement, has 2 or more clubs?

1. Figure out how to run one experiment (put it in a function, ex: ```experiment()```). 
2. Run the experiment a bunch of times (use a for loop!).
3. Calculate the proportion of times that the thing is true.

<!--
BEGIN QUESTION
name: q25
-->

### 1. Run one experiment

In [28]:
def experiment():
# BEGIN SOLUTION
    
    # draw 5 cards, without replacement
    hand = np.random.choice(ALL_CARDS, 5, replace=False)

    # count the number of clubs
    return number_of_suit(hand, 'clubs')

experiment()
# END SOLUTION

0

### 2. Run the experiment a bunch of times

Start by running it 1000 times

In [29]:
# BEGIN SOLUTION
results = np.array([])
times = 1_000

for time in np.arange(times):
    result = experiment()
    results = np.append(results, result)
# END SOLUTION

### 3. Calculate the probability

That is, what proportion of times did we see >= 2 clubs?

In [30]:
# BEGIN SOLUTION
np.count_nonzero(results >= 2) / times
# END SOLUTION

0.356

# Question 2.6

What is the probability that all of the cards in that hand are clubs?

<!--
BEGIN QUESTION
name: q26
-->

In [31]:
np.count_nonzero(results == 5) / times # SOLUTION

0.001

# Question 2.7

What is the probability of getting all red cards when drawing 5 cards without replacement?

HINT: Follow the same structure as we did before

<!--
BEGIN QUESTION
name: q27
-->

In [32]:
# BEGIN SOLUTION
def all_red(hand):
    for card in hand:
        if card.endswith('clubs') or card.endswith('spades'):
            return False
    return True
# END SOLUTION

In [33]:
# BEGIN SOLUTION
def experiment():
    # draw 5 cards, without replacement
    hand = np.random.choice(ALL_CARDS, 5, replace=False)

    # count the number of clubs
    return all_red(hand)

experiment()
# END SOLUTION

False

In [34]:
# BEGIN SOLUTION
results = np.array([])
times = 1_000

for time in np.arange(times):
    result = experiment()
    results = np.append(results, result)
# END SOLUTION

In [35]:
np.sum(results) / times # SOLUTION

0.025

In [36]:
grader.check_all()